In [33]:
from z3 import *
import Bool_utils as util
from itertools import combinations
import numpy as np

# Function to read the .dat file and return the parameters
def read_instance(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Parse m (number of couriers) and n (number of items)
    m = int(lines[0].strip())
    n = int(lines[1].strip())

    # Parse load limits
    load_limits = list(map(int, lines[2].strip().split()))

    # Parse item sizes
    item_sizes = list(map(int, lines[3].strip().split()))

    # Parse the distance matrix
    distance_matrix = []
    for i in range(n + 1):  # n+1 because the last row is the origin
        distance_matrix.append(list(map(int, lines[4 + i].strip().split())))

    return m, n, load_limits, item_sizes, distance_matrix

In [3]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

def plot_routes(distance_matrix, model, num_couriers, depot, assignments, route):
    # Create a graph
    G = nx.Graph()
    
    # Number of locations (objects + depot)
    num_locations = len(distance_matrix)
    
    # Add nodes (locations)
    for i in range(num_locations):
        G.add_node(i)
    
    # Add edges between all locations (with distance as edge weight)
    for i in range(num_locations):
        for j in range(i + 1, num_locations):
            G.add_edge(i, j, weight=distance_matrix[i][j])
    
    # Define colors for each courier
    courier_colors = ['r', 'g', 'b', 'y', 'c', 'm']  # List of colors (expand if more couriers)
    
    # Create a legend for the couriers
    courier_legend = {f"Courier {k+1}": courier_colors[k % len(courier_colors)] for k in range(num_couriers)}

    # Plot the routes for each courier
    plt.figure(figsize=(10, 8))
    pos = nx.spring_layout(G)  # Position the nodes using spring layout
    
    for k in range(num_couriers):
        courier_edges = []
        
        # Collect edges that belong to this courier
        for i in range(num_locations):
            for j in range(num_locations):
                if i != j and is_true(model.evaluate(route[i][j][k])):  # If this route is taken by courier k
                    courier_edges.append((i, j))
        
        # Draw the edges for this courier with specific color
        nx.draw_networkx_edges(G, pos, edgelist=courier_edges, edge_color=courier_colors[k % len(courier_colors)], width=2, alpha=0.7)
    
    # Draw nodes
    nx.draw_networkx_nodes(G, pos, node_size=500, node_color='lightgray', alpha=0.8)
    
    # Draw node labels (1-based indexing)
    labels = {i: str(i + 1) for i in range(num_locations)}  # Adding 1 to each node index
    nx.draw_networkx_labels(G, pos, labels=labels, font_size=12, font_weight='bold')
    
    # Draw edge labels (optional to show distance)
    edge_labels = {(i, j): distance_matrix[i][j] for i in range(num_locations) for j in range(i + 1, num_locations)}
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=8)
    
    # Add a legend for the couriers
    patches = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=10, label=label) 
               for label, color in courier_legend.items()]
    plt.legend(handles=patches, loc='upper left', title="Couriers")
    
    # Show the plot
    plt.title('Courier Routes')
    plt.axis('off')  # Hide axes
    plt.show()


In [39]:
def solve_mcp_1_with_bounds(file_path):
    # Read instance data from .dat file
    num_couriers, num_objects, capacities, weights, distance_matrix = read_instance(file_path)
    depot = num_objects  # Index for the depot (last element in distance matrix)
    
    # Convert distance matrix to NumPy array for easier manipulation
    Dnp = np.array(distance_matrix)

    # Compute Lower Bound (LB) and Loose Lower Bound (LLB)
    round_trips = [Dnp[i][depot] + Dnp[depot][i] for i in range(num_objects)]
    LB = max(round_trips)
    LLB = min(round_trips)

    # Compute Upper Bound (UB)
    UB = sum(np.max(Dnp[i, :]) for i in range(num_objects))

    print(f"Lower Bound (LB): {LB}")
    print(f"Loose Lower Bound (LLB): {LLB}")
    print(f"Upper Bound (UB): {UB}")

    # Variables, constraints, and solver setup
    load = [Int(f'load_{i}') for i in range(num_couriers)]
    assignments = [[Bool(f'assignments_{i}_{j}') for j in range(num_objects)] for i in range(num_couriers)]
    route = [[[Bool(f'route_{i}_{j}_{k}') for k in range(num_couriers)] for j in range(len(distance_matrix))] for i in range(len(distance_matrix))]
    visit_order = [[Int(f'visit_order_{j}_{k}') for j in range(num_objects + 1)] for k in range(num_couriers)]

    solver = Optimize()



    # Constraint: Each item should be assigned to exactly one courier
    for j in range(num_objects):
        solver.add(Or([assignments[k][j] for k in range(num_couriers)]))  # At least one assignment
        for k1, k2 in combinations(range(num_couriers), 2):
            solver.add(Or(Not(assignments[k1][j]), Not(assignments[k2][j])))  # At most one assignment

    # Load and capacity constraints for each courier
    for i in range(num_couriers):
        # Each load is the sum of weights assigned to the courier
        solver.add(load[i] == Sum([If(assignments[i][j], weights[j], 0) for j in range(num_objects)]))
        solver.add(load[i] <= capacities[i])


    # Symmetry-breaking constraints
    
    if np.all(Dnp==Dnp.T):
        for k in range(num_couriers):
            for p in range(k, num_couriers):
                if (load[k]==load[p]):
                    solver.add(util.compare(assignments[k],assignments[p],">="))
        for k in range(num_couriers):
            solver.add(util.compare([route[depot][i][k] for i in range(num_objects)],
                           [route[i][depot][k] for i in range(num_objects)], ">=")) 
    #limit on the second visit
#    for k in range(num_couriers-2):
 #       for i in range(num_objects):
  #          for j in range(i+1,num_objects+1):
   #             solver.add(Implies(route[0][j][k],Not(route[0][i][k+1])))

#crazy symmetry breaking

#    for i in range(num_couriers):  # For each courier
 #       for j in range(num_objects-1):
  #          for k in range(j+1, num_objects):  # Ensure smaller indices are visited first
   #             solver.add(Implies(route[j][k][i] == True, j < k))


#    for i in range(num_couriers-1):  # For consecutive couriers
 #       for j in range(num_objects):
  #          solver.add(Sum([assignments[i][j] for j in range(num_objects)]) <= Sum([assignments[i+1][j] for j in range(num_objects)]))

   # for i in range(num_couriers):
    #    for j in range(num_objects):
     #       for k in range(j+1, num_objects):
      #         if distance_matrix[j][k] == distance_matrix[k][j]:
       #             solver.add(Implies(route[j][k][i] == True, j < k))



    # Channeling constraints between assignments and routes
    for k in range(num_couriers):
        for j in range(num_objects):
            # If item `j` is assigned to courier `k`, courier `k` must visit `j`
            solver.add(Implies(assignments[k][j], Or([route[i][j][k] for i in range(num_objects + 1) if i != j])))

            # If there's a route to item `j` for courier `k`, item `j` must be assigned to courier `k`
            solver.add(Implies(Or([route[i][j][k] for i in range(num_objects + 1) if i != j]), assignments[k][j]))

    # Ensure each courier starts from and returns to the depot
    for k in range(num_couriers):
        # Courier `k` should start at the depot, with exactly one route from the depot to some other point
        solver.add(Sum([If(route[depot][j][k], 1, 0) for j in range(num_objects)]) == 1)

        # Courier `k` should return to the depot, with exactly one route to the depot from some other point
        solver.add(Sum([If(route[j][depot][k], 1, 0) for j in range(num_objects)]) == 1)

    # Flow control to prevent subtours and ensure continuous paths
    for k in range(num_couriers):
        solver.add(visit_order[k][depot] == 0)  # Depot is the starting point

        for j in range(num_objects + 1):
            # Ensure that each non-depot location (assigned location) has exactly one incoming and one outgoing route if visited
            if j != depot:
                solver.add(Sum([If(route[i][j][k], 1, 0) for i in range(num_objects + 1) if i != j]) == If(assignments[k][j], 1, 0))
                solver.add(Sum([If(route[j][i][k], 1, 0) for i in range(num_objects + 1) if i != j]) == If(assignments[k][j], 1, 0))

        # Enforce continuity in visit order if there is a route from i to j for courier k
        for i in range(num_objects + 1):
            for j in range(num_objects + 1):
                if i != j and j != depot:
                    solver.add(Implies(route[i][j][k], visit_order[k][j] == visit_order[k][i] + 1))

    # Objective: Minimize maximum distance traveled by any courier, within bounds
    max_distance = Int("max_distance")
    total_distance = [Int(f"total_distance_{i}") for i in range(num_couriers)]


    for k in range(num_couriers):
        # Sum distances for the route of each courier
        solver.add(total_distance[k] == Sum([If(route[i][j][k], distance_matrix[i][j], 0)
                                            for i in range(len(distance_matrix))
                                            for j in range(len(distance_matrix))]))
        solver.add(total_distance[k] <= max_distance)  # Ensure max_distance bounds each courier's total distance


    solver.minimize(max_distance)  # Minimize the maximum distance among all couriers

    # Add bounds as constraints
    solver.add(IntVal(LB) <= max_distance)
    solver.add(max_distance <= IntVal(UB))

    # Solve and output the solution
    if solver.check() == sat:
        model = solver.model()
        loads = [model.evaluate(load[i]) for i in range(num_couriers)]
        assignments_values = [[model.evaluate(assignments[i][j]) for j in range(num_objects)] for i in range(num_couriers)]
        distances = [model.evaluate(total_distance[i]) for i in range(num_couriers)]
        print("Loads per courier:", loads)
        print("Assignments:", assignments_values)
        print("Distances per courier:", distances)
        print("Max distance:", model.evaluate(max_distance))

        for k in range(num_couriers):
            print(f"Courier {k + 1}:")
            for i in range(len(distance_matrix)):
                for j in range(len(distance_matrix)):
                    if is_true(model.evaluate(route[i][j][k])):
                        print(f"  Point {i + 1} -> Point {j + 1}")
        
        # Plot routes (this assumes the plot_routes function exists)
        plot_routes(distance_matrix, model, num_couriers, depot, assignments, route)
    else:
        print("No solution found.")


In [41]:
solve_mcp_1_with_bounds(file_path="/Users/shariqansari/Documents/CDMO/instances/inst12.dat")

Lower Bound (LB): 346
Loose Lower Bound (LLB): 12
Upper Bound (UB): 18108
